In [1]:
import torch
import torch.nn as nn
from rfft_lib import rfft_audio
import onnx
import math

In [2]:
audio_path = "/home/satya-tt0492/Documents/TT/ex-7/audio/large1.wav"

In [3]:
audio = rfft_audio.load_audio(audio_path)
print(audio)

tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [4]:
audio.shape

torch.Size([7674624])

In [5]:
def next_power_of_2_calc(n):
    if n <= 1:
        return 1
    return 2 ** (math.ceil(math.log2(n)))

In [6]:
N = audio.shape[0]
    
# Pad to the next power of 2 if necessary
if (N & (N - 1)) != 0:
    next_power_of_2 = next_power_of_2_calc(N)
    padding_size = next_power_of_2 - N
    audio = torch.cat([audio, torch.zeros(padding_size)])
    N = next_power_of_2

In [7]:
audio

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [8]:
audio.shape

torch.Size([8388608])

In [9]:
class RFFTModel(nn.Module):
    def __init__(self):
        super(RFFTModel,self).__init__()
    
    def forward(self, audio):
        rfft_magnitude = rfft_audio.apply_rfft(audio)
        return rfft_magnitude

In [10]:
model = RFFTModel()

In [11]:
onnx_program = torch.onnx.dynamo_export(
    model,
    audio,
)

/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/torch/onnx/_internal/exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/torch/onnx/_internal/fx/onnxfunction_dispatcher.py:529:

In [12]:
onnx_program.save("rfft_model.onnx")

In [13]:
onnx_model = onnx.load("rfft_model.onnx")
onnx.checker.check_model(onnx_model)

In [14]:
import onnxruntime

In [15]:
ort_session = onnxruntime.InferenceSession("rfft_model.onnx")

In [16]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [17]:
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(audio)

In [18]:
onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}
onnx_output = ort_session.run(None,onnxruntime_input)

In [19]:
x = onnx_output[0]

In [20]:
y = abs(torch.fft.rfft(audio)).numpy()

In [21]:
import numpy as np

In [22]:
np.max(np.abs(x - y))

0.0234375

In [24]:
y[:10]

array([736.37964 , 298.888   ,  98.68868 ,  24.334352, 144.6005  ,
       158.98279 , 178.61069 ,  41.855766, 115.83441 ,  27.6268  ],
      dtype=float32)

In [25]:
x[:10]

array([736.37964 , 298.88803 ,  98.68851 ,  24.334215, 144.60045 ,
       158.98291 , 178.61075 ,  41.855846, 115.83447 ,  27.626785],
      dtype=float32)